<a href="https://colab.research.google.com/github/Saranyanv/test/blob/master/Reducing%20Commercial%20Aviation%20Fatalities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**REDUCING COMMERCIAL AVIATION FATALITIES**

*https://www.kaggle.com/c/reducing-commercial-aviation-fatalities/data*

In [0]:
import os 
import numpy as np
import zipfile
from tqdm import tqdm
from matplotlib import pyplot as plt
from pandas import DataFrame
import pandas as pd
import seaborn as sns
RANDOM_SEED = 42

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!ls -l "drive/My Drive/MyWork/Kaggle/Reducing Commercial Aviation Fatalities"

total 439092
-rw------- 1 root root 449629430 Mar 17 03:41 train.csv.zip


In [10]:
!unzip "drive/My Drive/MyWork/Kaggle/Reducing Commercial Aviation Fatalities/train.csv.zip"

Archive:  drive/My Drive/MyWork/Kaggle/Reducing Commercial Aviation Fatalities/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [11]:
!ls -lrt /content/

total 1205840
---------- 1 root root 1234764001 Dec 14 23:11 train.csv
drwxr-xr-x 1 root root       4096 Apr  4 20:20 sample_data
drwx------ 3 root root       4096 Apr 11 01:21 drive


In [0]:
#train_df = pd.read_csv('train.csv', delim_whitespace=False, names=('id', 'crew', 'experiment', 'time', 'seat', 'eeg_fp1', 'eeg_f7', 'eeg_f8', 'eeg_t4', 'eeg_t6', 'eeg_t5', 'eeg_t3', 'eeg_fp2', 'eeg_o1', 'eeg_p3', 'eeg_pz', 'eeg_f3', 'eeg_fz', 'eeg_f4', 'eeg_c4', 'eeg_p4', 'eeg_poz', 'eeg_c3', 'eeg_cz', 'eeg_o2', 'ecg', 'r', 'gsr', 'event'))
train_df = pd.read_csv('train.csv', header=0, dtype={'crew': np.int32, 'experiment' :str, 'time': np.float64, 'seat': np.int32, 'eeg_fp1': np.float64, 'eeg_f7': np.float64, 'eeg_f8': np.float64, 'eeg_t4': np.float64, 'eeg_t6': np.float64, 'eeg_t5': np.float64, 'eeg_t3': np.float64, 'eeg_fp2': np.float64, 'eeg_o1': np.float64, 'eeg_p3': np.float64, 'eeg_pz': np.float64, 'eeg_f3': np.float64, 'eeg_fz': np.float64, 'eeg_f4': np.float64, 'eeg_c4': np.float64, 'eeg_p4': np.float64, 'eeg_poz': np.float64, 'eeg_c3': np.float64, 'eeg_cz': np.float64, 'eeg_o2': np.float64, 'ecg': np.float64, 'r': np.float64, 'gsr': str, 'event': str})

In [13]:
len(train_df)

4867421

In [14]:
train_df.head()

,crew,experiment,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,eeg_t5,...,eeg_c4,eeg_p4,eeg_poz,eeg_c3,eeg_cz,eeg_o2,ecg,r,gsr,event
0,1,CA,0.011719,1,-5.28545,26.775801,-9.527310,-12.793200,16.717800,33.737499,...,37.368999,17.437599,19.201900,20.5968,-3.95115,14.507600,-4520.0,817.705994,388.829987,A
1,1,CA,0.015625,1,-2.42842,28.430901,-9.323510,-3.757230,15.969300,30.443600,...,31.170799,19.399700,19.689501,21.3547,1.33212,17.750200,-4520.0,817.705994,388.829987,A
2,1,CA,0.019531,1,10.67150,30.420200,15.350700,24.724001,16.143101,32.142799,...,-12.012600,19.396299,23.171700,22.4076,1.53786,22.247000,-4520.0,817.705994,388.829987,A
3,1,CA,0.023438,1,11.45250,25.609800,2.433080,12.412500,20.533300,31.494101,...,18.574100,23.156401,22.641199,19.3367,2.54492,18.998600,-4520.0,817.705994,388.829987,A
4,1,CA,0.027344,1,7.28321,25.942600,0.113564,5.748000,19.833599,28.753599,...,6.555440,22.754700,22.670300,20.2932,1.69962,22.812799,-4520.0,817.705994,388.829987,A


In [0]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [87]:
train_df.event.value_counts()

A    2848809
C    1652686
D     235329
B     130597
Name: event, dtype: int64

In [0]:
class_variance=train_df.groupby('event').agg({'eeg_fp1': np.var, 'eeg_f7': np.var, 'eeg_f8': np.var, 'eeg_t4': np.var, 'eeg_t6': np.var, 'eeg_t5': np.var, 'eeg_t3': np.var
, 'eeg_fp2': np.var, 'eeg_o1': np.var, 'eeg_p3': np.var, 'eeg_pz': np.var, 'eeg_f3': np.var, 'eeg_fz': np.var, 'eeg_f4': np.var
, 'eeg_c4': np.var, 'eeg_p4': np.var, 'eeg_poz': np.var, 'eeg_c3': np.var, 'eeg_cz': np.var, 'eeg_o2': np.var, 'ecg': np.var, 'r': np.var})

In [0]:
#fig, ax = plt.subplots(figsize=(14, 8))
#sns.countplot(x='event', data=train_df, palette='hls')

In [95]:
train_df.isnull().sum().sum()

0

In [0]:
#data cleaning start with matrix of histogram with seaborn